### Importing Libaries ###

In [26]:
#Library imports
import re
import os
import time
import math
import random
import itertools
import warnings
import numpy as np
from numpy import array
import pandas as pd
from pandas import datetime
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from pylab import rcParams
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn import svm
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import pmdarima as pm
from pmdarima.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

warnings.filterwarnings("ignore")
#Notebook arguments
%matplotlib inline

# Importing Data

In [2]:
# Importing cleaned, timstamped and indexed daily index value data 
vix_arima = pd.read_pickle("vix_clean.pickle")

In [3]:
# Importing cleaned, timstamped and indexed daily sp500 value data 
vix_sp500 = pd.read_pickle('sp500_vols_clean.pickle')

In [4]:
# Sanity Check
vix_arima.head()

,adjclose
Date,
2010-06-23,26.910000
2010-06-24,29.740000
2010-06-25,28.530001
2010-06-28,29.000000
2010-06-29,34.130001


In [5]:
# Sanity Check
vix_sp500.head()

,Volume
Date,
2010-06-23,4526150000
2010-06-24,4814830000
2010-06-25,5128840000
2010-06-28,3896410000
2010-06-29,6136700000


In [ ]:
# concat two dataframes 



In [6]:
#Sanity Check
vix_arima.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2518 entries, 2010-06-23 to 2020-06-23
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   adjclose  2518 non-null   float64
dtypes: float64(1)
memory usage: 39.3 KB


In [7]:
vix_arima.head()

,adjclose
Date,
2010-06-23,26.910000
2010-06-24,29.740000
2010-06-25,28.530001
2010-06-28,29.000000
2010-06-29,34.130001


## Vanilla LSTM 

In [27]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
raw_seq = vix_arima['adjclose']
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[26.91     29.74     28.530001] 29.0
[29.74     28.530001 29.      ] 34.130001
[28.530001 29.       34.130001] 34.540001000000004
[29.       34.130001 34.540001] 32.860001000000004
[34.130001 34.540001 32.860001] 30.120001000000002
[34.540001 32.860001 30.120001] 29.65
[32.860001 30.120001 29.65    ] 26.84
[30.120001 29.65     26.84    ] 25.709999
[29.65     26.84     25.709999] 24.98
[26.84     25.709999 24.98    ] 24.43
[25.709999 24.98     24.43    ] 24.559998999999998
[24.98     24.43     24.559999] 24.889999
[24.43     24.559999 24.889999] 25.139999
[24.559999 24.889999 25.139999] 26.25
[24.889999 25.139999 26.25    ] 25.969998999999998
[25.139999 26.25     25.969999] 23.93
[26.25     25.969999 23.93    ] 25.639999
[25.969999 23.93     25.639999] 24.629998999999998
[23.93     25.639999 24.629999] 23.469998999999998
[25.639999 24.629999 23.469999] 22.73
[24.629999 23.469999 22.73    ] 23.190001000000002
[23.469999 22.73     23.190001] 24.25
[22.73     23.190001 24.25    ] 24.129998

[19.41 20.34 19.6 ] 16.48
[20.34 19.6  16.48] 15.72
[19.6  16.48 15.72] 16.07
[16.48 15.72 16.07] 18.66
[15.72 16.07 18.66] 14.71
[16.07 18.66 14.71] 13.48
[18.66 14.71 13.48] 13.04
[14.71 13.48 13.04] 13.16
[13.48 13.04 13.16] 13.33
[13.04 13.16 13.33] 13.42
[13.16 13.33 13.42] 13.2
[13.33 13.42 13.2 ] 13.09
[13.42 13.2  13.09] 13.31
[13.2  13.09 13.31] 13.41
[13.09 13.31 13.41] 13.65
[13.31 13.41 13.65] 13.75
[13.41 13.65 13.75] 13.28
[13.65 13.75 13.28] 12.93
[13.75 13.28 12.93] 13.27
[13.28 12.93 13.27] 12.67
[12.93 13.27 12.67] 13.91
[13.27 12.67 13.91] 12.9
[12.67 13.91 12.9 ] 12.53
[13.91 12.9  12.53] 12.82
[12.9  12.53 12.82] 12.52
[12.53 12.82 12.52] 12.37
[12.82 12.52 12.37] 12.19
[12.52 12.37 12.19] 13.1
[12.37 12.19 13.1 ] 13.39
[12.19 13.1  13.39] 13.4
[13.1  13.39 13.4 ] 12.66
[13.39 13.4  12.66] 12.26
[13.4  12.66 12.26] 12.79
[12.66 12.26 12.79] 12.81
[12.26 12.79 12.81] 12.98
[12.79 12.81 12.98] 13.7
[12.81 12.98 13.7 ] 14.23
[12.98 13.7  14.23] 14.55
[13.7  14.23 14.5

[24.389999 22.73     20.950001] 17.860001
[22.73     20.950001 17.860001] 18.940001000000002
[20.950001 17.860001 18.940001] 20.700001
[17.860001 18.940001 20.700001] 18.700001
[18.940001 20.700001 18.700001] 16.6
[20.700001 18.700001 16.6     ] 15.57
[18.700001 16.6      15.57    ] 15.74
[16.6  15.57 15.74] 16.91
[15.57 15.74 16.91] 16.08
[15.74 16.91 16.08] 17.290001
[16.91     16.08     17.290001] 18.209999
[16.08     17.290001 18.209999] 20.700001
[17.290001 18.209999 20.700001] 19.34
[18.209999 20.700001 19.34    ] 20.59
[20.700001 19.34     20.59    ] 24.99
[19.34 20.59 24.99] 27.01
[20.59 24.99 27.01] 24.299999
[24.99     27.01     24.299999] 22.469998999999998
[27.01     24.299999 22.469999] 25.219998999999998
[24.299999 22.469999 25.219999] 23.950001
[22.469999 25.219999 23.950001] 27.02
[25.219999 23.950001 27.02    ] 26.049999
[23.950001 27.02     26.049999] 27.59
[27.02     26.049999 27.59    ] 26.690001000000002
[26.049999 27.59     26.690001] 22.34
[27.59     26.690001 22

[21.25     19.98     18.139999] 20.1
[19.98     18.139999 20.1     ] 22.48
[18.139999 20.1      22.48    ] 20.799999
[20.1      22.48     20.799999] 21.52
[22.48     20.799999 21.52    ] 18.9
[20.799999 21.52     18.9     ] 19.02
[21.52 18.9  19.02] 18.49
[18.9  19.02 18.49] 18.790001
[19.02     18.49     18.790001] 18.07
[18.49     18.790001 18.07    ] 16.440001000000002
[18.790001 18.07     16.440001] 20.74
[18.07     16.440001 20.74    ] 21.190001000000002
[16.440001 20.74     21.190001] 23.23
[20.74     21.190001 23.23    ] 22.639999
[21.190001 23.23     22.639999] 21.76
[23.23     22.639999 21.76    ] 21.459999
[22.639999 21.76     21.459999] 20.65
[21.76     21.459999 20.65    ] 21.629998999999998
[21.459999 20.65     21.629999] 24.52
[20.65     21.629999 24.52    ] 25.58
[21.629999 24.52     25.58    ] 25.58
[24.52 25.58 25.58] 28.379998999999998
[25.58     25.58     28.379999] 30.110001
[25.58     28.379999 30.110001] 36.07
[28.379999 30.110001 36.07    ] 30.41
[30.110001 36.07

In [29]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [30]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [31]:
# fit model
model.fit(X, y, epochs=200, verbose=0)
